## Automated Extraction of Leaf Area from Photos

##### Take photos of leaves with any camera*, populate a folder, and quickly leaf extract data

*on white background of known dimensions

To answer some important plant biology, horticultural or agronomic research questions it can be important to accurately measure the area of many leaves. This labor intensive process has previously limited the size of datasets used in publications, and a faster solution would vastly improve the size and quality of these datasets.

The following code represents a fast, reliable and simple framework for rapidly extracting leaf area from large numbers of images captured with a camera.

In [1]:
# import packages
import os
import cv2
import skimage
import pandas as pd
import numpy as np
import imutils
from imutils.perspective import four_point_transform
from imutils import contours
from skimage import io
from PIL import Image

In [2]:
# define functions
def getListOfFiles(dirName):
    # create a list of file and sub directories names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

### Set some variables

Edit the folder name and paper size to appropriate values.

Set paper or white background size using any units you prefer.

In [3]:
# set folder name (including path from this jupyter notebook)
folder = "photos"

# set paper or white background size
paper_size = 0.06032246 # use units of m^2

# build list of image files (do not edit)
file_list = getListOfFiles(folder)
if 'photos/.DS_Store' in file_list:
    file_list.remove('photos/.DS_Store')

### Loop over all the photos in folder

Extract leaf area, then add file name, date of capture and leaf area to a dataframe

In [62]:
# reset counter
z = 0

# loop over all images in the list of image files
while z < len(file_list):

    # load the image
    image = cv2.imread(file_list[z])

    # resize image
    image = imutils.resize(image, height = 1000)

    # convert resized image to grayscale, blur it slightly, then find edges
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 75, 200)
    
    # apply Otsu's thresholding method to create a binarized version, too
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # find contours in the edge map, keeping only the largest ones
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True) # approximate the contour
        if len(approx) == 4: # if approx contour has four points, then assume that we found paper
            screenCnt = approx
            break
            
    # apply the four point transform to obtain a top-down view of the paper area only         
    warped = four_point_transform(thresh, screenCnt.reshape(4, 2))  

    # extract leaf area
    n_white = cv2.countNonZero(warped)
    height, width = warped.shape
    n_total = height * width
    leaf_area = (n_white / n_total) * paper_size

    # extract creation time from image metadata
    imageName = file_list[z]
    creation_time = Image.open(imageName)._getexif()[36867]

    if z == 0:
            
        # initialize list of lists
        data = [[creation_time[:19], leaf_area]]

        # Create the pandas DataFrame
        dict = {'File Name':[],'Date Time':[],'Leaf Area':[]}
        df = pd.DataFrame(dict)
        
        z += 1

    else:
        # append onto existing results
        dict = {'File Name': [imageName[7:20]],'Date Time': [creation_time[:19]], 'Leaf Area': [leaf_area]}
        new_row = pd.DataFrame(dict)
        df = pd.concat([new_row, df], ignore_index = True)

        z += 1

### Print the dataframe with image datetime and leaf area

In [63]:
display(df)

,File Name,Date Time,Leaf Area
0,IMG_0671.jpeg,2023:05:25 09:08:05,0.002756
1,IMG_0734.jpeg,2023:06:06 22:37:42,0.001303
2,IMG_0763.jpeg,2023:06:06 22:44:08,0.001504
3,IMG_0775.jpeg,2023:06:06 22:46:04,0.001461
4,IMG_0667.jpeg,2023:05:25 09:04:48,0.000456
...,...,...,...
100,IMG_0740.jpeg,2023:06:06 22:39:33,0.001865
101,IMG_0737.jpeg,2023:06:06 22:38:48,0.001382
102,IMG_0672.jpeg,2023:05:25 09:08:18,0.001919
103,IMG_0760.jpeg,2023:06:06 22:43:35,0.001316


## Save the dataframe

In [64]:
# set folder
folder = 'results/'

# set filename
filename = 'name.csv'

# save file
df.to_csv(folder + filename)

**Authors:**
Matthew Jenkins, 
Dylan Lenczewski-Jowers, 
Fabiola Chavez Lamas